# K-Prototypes Tech Talk

The summary below closely follows the original paper, [Huang 1998](https://link.springer.com/article/10.1023/A:1009769707641).

## Outline
* Unsupervised machine learning and what does it mean to cluster data
    * How does regular K-means work?
    * Other clustering algos
* What are its advantages and limitations
* Why is using standard distance metric problematic?
* K-prototypes
* What libraries are available?
* Application to Colombia vulnerability index

## References

* Python implementation
    * https://pypi.org/project/kmodes/
    * https://github.com/nicodv/kmodes
* Other articles and notebooks using k-prototypes
    * https://www.kaggle.com/rohanadagouda/unsupervised-learning-using-k-prototype-and-dbscan
    * https://towardsdatascience.com/the-k-prototype-as-clustering-algorithm-for-mixed-data-type-categorical-and-numerical-fe7c50538ebb

* Data for this demo
    * https://www.kaggle.com/giovamata/airlinedelaycauses
* [Unsupervised clustering with mixed categorical and continuous data](https://www.tomasbeuzen.com/post/clustering-mixed-data/)
    
https://www.kaggle.com/ashydv/bank-customer-clustering-k-modes-clustering






## How does K-means work

### K-means algorithm
* Initialize with a random set of cluster vectors (these are m-dimensional vectors where m is the number of features in the dataset)
* In a loop:
    * Allocate each point in the data to the nearest cluster based on the Euclidean distance
    * For each cluster, update the cluster vector by calculating the mean of all the records assigned to that cluster (average of each component across all vectors belonging to the same cluster).
    * Repeat the first step by checking and reallocating each data point based on the new cluster vectors
    * Repeat until desired number of iteration is reached or record changes cluster
    
Here's a visualization of the algorithm
    
    
![kmeans_algo](./imgs/kmeans_algo.png)
[Source](https://en.wikipedia.org/wiki/K-means_clustering)

## Why K-means shouldn't be used with categorical variables

Categorical variables are data that takes one of a limited and usually fixed number of possible values. If they can be ordered, then it is called `ordinal`. If it cannot be ordered, then it is called `nominal`. Here are some examples:

    * ordinal: risk rating, rank, day of the week
    * nominal: political party, region of residence, occupation

There are several ways these variables are used in models. For unsupervised ML, some approaches include one-hot encoding, label encoding, feature aggregation, or excluding the feature from the model.

For one-hot encoding, the categorical is converted to several binary features each of which representing a given category in the original feature. This works when there are few categories, but become a computational burden when categories exceeds hundres or thousands of unique values (e.g. zip codes).

Label encoding only works for ordinal data, but even in such case it would require that the encoded feature follows the same ordering as the original values in the raw feature.

In the case of a categorical feature with high cardinality, one solution is to aggregate them. For example, zip codes can be aggregated to a region containing many zip codes. Following the aggregation, one can then apply one-hot encoding.

When the approaches above fails or is infeasible to apply, the data scientist may exclude categorical feature all together from the model.

## Why is using standard distance metric problematic for categorical data?


For numerical data, it is natural to define the center of mass for a collection of points using common distance metrics, such as the Euclidean distance. Say you have three points in your dataset containing annual income: `[$100k, $45k, $65k]`. One metric for the center of mass is just the average, in this case, `$70k`. 

But instead of income, suppose you had categories of hair color: `[blond, red, black]`. How can one define center of mass in this case?


## K-prototypes

K-prototypes combines k-means and k-modes into to be able to handle both numeric and cateogircal variables. The k-modes algorithm differs from k-means in the following way:

- Instead of Euclidean distance, use an alternative dissimilarity measure for categorical objects
- Instead of mean of a set of points to define a cluster centroid, use the mode where a frequency-based approach to find modes for the clusters


### K-modes

Dissimilarity measure for categorical attributes

```
x1 = [finance, married, golf] 
	is similar to 
x2 = [finance, married, squash] 
	but dissimilar to 
x3 = [healthcare, single, soccer]
```

More formally,

<img src="./imgs/dissimilarity_measure.png" alt="drawing" width="250"/>

Use the mode to represent the "center" of a set of vectors containing only categoricals. The mode itself need not be part of the original set. The formal definition for the mode is a vector Q such that this Q minimizes the distance between itself and the other vectors in the set using the dissimilarity measure above.

More formally

<img src="./imgs/cat_mode.png" alt="drawing" width="500"/>

Now that we have an approach for defining how close two vectors contaning categorical values are and a technique for determining the center of mass for a set of categorical-valued vectors, we can use a similar algorithm as k-means to find the clusters.

The k-modes algorithm:

* Select the k initial modes, one for each cluster
* In a loop, 
    * Allocate each point in the data to the nearest cluster based on the dissimilarity measure defined above
    * Update the mode of the cluster based on the set of points allocated to it
    * Repeat the first step by checking and reallocating each data point based on the new cluster modes
    * Repeat until no record changes cluster or the desired number of iteration is reached

### Combining k-means and k-modes

Combining k-means and k-modes requires a function that combines both the Euclidean distance metric and the dissimilarity measure function. Formally, the function is formulated as 

<img src="./imgs/combined_euc_dissm.png" alt="drawing" width="300"/>

Similar to the k-means and k-modes algorithm above, the optimization problems boils down to finding the cluster vectors (that now contain both numeric and categorical values) such that the sum of the euclidean distance metric and dissimilarity measure across all k cluster vectors is minimized.

In [2]:
# Importing required packages
import numpy as np
import pandas as pd 
import random
from kmodes.kprototypes import KPrototypes
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

# German Credit Data

Source
    * https://towardsdatascience.com/clustering-datasets-having-both-numerical-and-categorical-variables-ed91cdca0677
    * https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/

In [33]:
col_names = [
    "chk_acct", 
    "duration", 
    "credit_his", 
    "purpose", 
    "amount", 
    "saving_acct", 
    "present_emp", 
    "installment_rate", 
    "sex", 
    "other_debtor",                   
    "present_resid", 
    "property", 
    "age", 
    "other_install", "housing", 
    "n_credits", 
    "job", 
    "n_people", 
    "telephone", 
    "foreign", 
    "response"
]

df = pd.read_csv('./data/german/german.data',
                 sep=' ', 

                 header=None,
                 names=col_names)

In [32]:
df.head().T

,0,1,2,3,4
chk_acct,A11,A12,A14,A11,A11
duration,6,48,12,42,24
credit_his,A34,A32,A34,A32,A33
purpose,A43,A43,A46,A42,A40
amount,1169,5951,2096,7882,4870
saving_acct,A65,A61,A61,A61,A61
present_emp,A75,A73,A74,A74,A73
installment_rate,4,2,2,2,3
sex,A93,A92,A93,A93,A93
other_debtor,A101,A101,A101,A103,A101
